# A simple example on how to use `scopes`

In this tutorial you will learn the basic functionality of scopes and how to set everything up to use it.

## Topics

- Construct the "Night" object to designate the specific night of the year for which the schedule is to be created.
- Establish the observing programs, outlining their specific objectives and parameters.
- Determine the merits to be utilized, ensuring they align with the objectives of the observing programs.
- Create the "Target" objects, assigning appropriate merits based on the unique requirements of each target.
- Create the "Observation" objects, detailing the specifics of each observation.
- Compile a preliminary night schedule using the created observations, forming an organized plan for the designated night.

In [2]:
from scopes.scheduler_components import Night, Program, Merit, Target, Observation
from scopes import merits
from scopes import scheduler

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from datetime import date, timedelta
from astropy.coordinates import SkyCoord
import astroplan

### Observer and Night

We will start by defining the Observer, that is where in the world our telescope is located, and for which night we want to create a schedule. This is done using the Observer object from the astroplan package. For this example we will consider a telescope at the La Silla Observatory in Chile.

We then create the Night object for which night we want to create the schedule. This is done by defining the date, within which twilights observations should be considered

In [3]:
# Define observer location
observer = astroplan.Observer.at_site("lasilla")

# Define the night
night = Night(date(2023, 11, 14), "nautical", observer)

### Programs

SCOPES allows to define different observing programs. This is widely the case in many telescopes where different groups use the telescope for different purposes and thus each group or person has their own observing program with their own set of targets and scientific priorities.

These are defined with the Program object where the parameters to be given are the program ID, name of the isntrument to be used, the proprotion of the total time that is allocated to this program, and optionally with what color this program will be plotted.

In [5]:
# Define color pallette for plotting
color_pallette = iter([mcolors.rgb2hex(color) for color in plt.get_cmap("Set2").colors])

# Lets use "CAR" and "BIKE" as our two example instruments
prog1 = Program("prog1", "CAR", 0.1, 1, plot_color=next(color_pallette))
prog2 = Program("prog2", "CAR", 0.1, 2, plot_color=next(color_pallette))
prog3 = Program("prog3", "CAR", 0.3, 2, plot_color=next(color_pallette))
prog4 = Program("prog4", "BIKE", 0.2, 3, plot_color=next(color_pallette))
prog5 = Program("prog5", "BIKE", 0.3, 2, plot_color=next(color_pallette))

## Merits

Now we will define the set of merits we will be using. Some standard merit functions can be found in the `scopes.merits` module, but custom ones can be defined as well.

The Merit class takes three mandatory arguments, `name`, `func`, and `merit_type`. `func` is the function that actually computes the merit from an observation, `merit_type` tells the scheduler how to use the merit in the rank function, i.e. if its a fariness, veto, or efficiency merit. Then the optional `paramteres` arguments gives the function any additional keyword arguments for that merit. This allows to create merits that use the same merit function but with a different set of parameters.

In [6]:
# We will start with the basic observability merits like limits on the airmass, altitude, and time of night.

# Lets define a merit for the airmass setting a maximum of 1.8
airmass_merit = Merit(
    "Airmass", merits.airmass, merit_type="veto", parameters={"max": 1.8}
)
# Lets define a merit for the altitude. These are usually set by the hardware limits of the telescope.
# In this exmaple we will use a minimum of 20 and a maximum of 87 degrees (to avoid the zenith
# which can cause tracking issues in telescopes with an altazimuth mount)
altitude_merit = Merit(
    "Altitude", merits.altitude, merit_type="veto", parameters={"min": 20, "max": 87}
)
# Lets define a merit for the time of night. These limits are used from the Night object we created above.
at_night_merit = Merit("AtNight", merits.at_night, merit_type="veto")

# The Culmination merit is used to ensure observations are done close to the culmination of the target in the sky.
culmapping_merit = Merit(
    "CulMapping", merits.culmination_mapping, merit_type="efficiency"
)

# Lastly, we will deifne a fairness merit for the time share. This merit ensures that programs
# respect the time share they are given.
timeshare_merit = Merit("Timeshare", merits.time_share, merit_type="fairness")

## Targets and Observations

Next we will define the targets to be observed by creating Target objects and then Observation Objects

In [ ]:
# Generate random star coordinates
np.random.seed(0)
ra = np.random.uniform(0, 360, size=10)
dec = np.random.uniform(-90, 90, size=10)

# Create Target objects
targets = []
for i in range(len(ra)):
    coord = SkyCoord(ra[i], dec[i], unit="deg")
    target = Target(coord)
    targets.append(target)

In [ ]:
obs = []
for i in range(20):
    obs.appen(Observation())